In [63]:
import pandas as pd
import numpy as np
from textFunctions import *
from datetime import datetime
import gc
from nltk import FreqDist
import lda
from ast import literal_eval
import os
import glob
import re
import string
from __future__ import print_function
import funcy as fp
import gensim
from gensim import models
from gensim.utils import tokenize
# from gensim.corpora import Dictionary, MmCorpus
from gensim import corpora
from sklearn.manifold import TSNE
import nltk
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from textFunctions import *
pd.set_option('display.max_rows', 999)

In [38]:
books = glob.glob('./texts/*.txt')
d = list()
for book_file in books:
    with open(book_file, encoding='utf-8') as f:
        book = os.path.basename(book_file.split('.')[0])
        d.append(pd.DataFrame({'book': book, 'text': f.readlines()}))
doc = pd.concat(d)
doc.head()

,book,text
0,,The Discourse on the Arousing of Mindfulness d...
1,,\n
2,,"Mindfulness is a process, an event and an aris..."
3,,\n
4,,For instance in association with Right Underst...


In [40]:
sati=doc[doc['text'] != '\n']

In [41]:
suttas=pd.read_csv('./texts/suttas.csv')

In [42]:
suttas.rename(columns={'Unnamed: 0':'book'},inplace=True)

In [48]:
li=[sati,suttas]
doc=pd.concat(li)

In [49]:
len(doc)==len(sati)+len(suttas)

True

In [50]:
doc.head()

,book,text
0,,The Discourse on the Arousing of Mindfulness d...
2,,"Mindfulness is a process, an event and an aris..."
4,,For instance in association with Right Underst...
6,,But the intuitive or rational role does not pr...
8,,Mindfulness as memory is indicated by such ter...


In [64]:
doc['token'] = doc['text'].astype(str).apply(lambda x: tokenize(x))

In [65]:
doc.head()

,book,text,token
0,,The Discourse on the Arousing of Mindfulness d...,"[[The, Discourse, on, the, Arousing, of, Mindf..."
2,,"Mindfulness is a process, an event and an aris...","[[Mindfulness, is, a, process,, an, event, and..."
4,,For instance in association with Right Underst...,"[[For, instance, in, association, with, Right,..."
6,,But the intuitive or rational role does not pr...,"[[But, the, intuitive, or, rational, role, doe..."
8,,Mindfulness as memory is indicated by such ter...,"[[Mindfulness, as, memory, is, indicated, by, ..."


In [66]:
def most_used(art, ngrams=1, mod=None, month='', site=''):
    '''Returns dict with counts, and the number of articles in a given dataframe.
    
    Parameters:
    - art: dataframe with articles
    - ngrams: n of n-grams (default: 1, words)
    - mod: dir to save files with articles (None if no saving necessary)
    - month: month for filename
    - site: site for filename
    '''
    word_dict = {} # save to it later
    print(art.shape[0], month, site) # number of articles
    if art.shape[0] == 0:
        return {}, 0 # we don't divide by it later; continue
    art['text'].fillna('', inplace=True) # get rid of nans, just in case. There should be none
    # and there are none in articles, but reddit doesn't work without this line
    time_start = datetime.now()
    art_sh = art[['text']].copy() # we don't need more columns
    del art # get rid of original df
    gc.collect() # clean memory, save RAM
    div = art_sh.shape[0]
    print(art_sh.shape[0]) # number of articles without nans
    
    # clean texts a bit using regexp
    # later we split by space (in function imported from textFunctions), so it's necessary
    art_sh['text'] = art_sh['text'].apply(lambda x: re.sub('\n', ' ', x))
    art_sh['text'] = art_sh['text'].apply(lambda x: re.sub('\t', ' ', x))

    # tokenize
    bigrams = False
    if ngrams == 2:
        bigrams = True
    art_sh['text'] = art_sh['text'].apply(lambda x: tokenize(x, bigrams=bigrams, words=True, lang='english'))

    # cleaning, quite slow, but necessary
    # this function is actually quite minimal to save time, if you need something else, you may want more preprocessing
    art_sh['text'] = art_sh['text'].apply(lambda x: cleaning(x, removePunctuation=True, removeNumbers=True, toLower=True, removeSpaces=True, stopwords=True, decodeDiacritics=False, shortwords=True)) 

    # frequency
    fdist = FreqDist([i for sublist in art_sh['text'].tolist() for i in sublist])
    x = dict(fdist)
    del fdist
    gc.collect()
    
    # we may need chunks later
    # word_dict = {k: x.get(k, 0) + word_dict.get(k, 0) for k in set(x) | set(word_dict)}
    
    # save tokenized articles
    if mod is not None:
        art_sh[['text']].to_csv(mod + '' + str(ngrams) + '.csv')

    print(datetime.now() - time_start)
    del art_sh
    gc.collect()
    return x, div

In [67]:
res = './res/sutta/'
for n in [1,2]:
    word_dfs_all = pd.DataFrame()
    # call the function defined above
    word_dict, div = most_used(doc, ngrams=n, mod=res)
            # convert result to df
    if div == 0:
        continue # avoid division by 0 and assigning column names to empty dataframes
    word_df = pd.DataFrame.from_dict(word_dict, orient='index')
    del word_dict
    gc.collect()
    # we need counts and frequencies
    word_df.columns = ['count']
    word_df['freq'] = word_df['count'] / div
    word_dfs_all = word_df.copy()
    print(word_dfs_all.shape)
    del word_df
    gc.collect()
    word_dfs_all.to_csv(res + 'freq_' + str(n) + '-all.csv')
    del word_dfs_all # we won't need it later
    gc.collect()

3712  
3712
0:00:19.356252
(19466, 2)
3712  
3712
0:00:23.348895
(173210, 2)


In [73]:
freq=pd.read_csv('./res/sutta/freq_1-all.csv')
freq.rename(columns={'Unnamed: 0': 'word'},inplace=True)

In [74]:
freq.sort_values('count',ascending=False).to_csv('sutta_word_count.csv')

In [91]:
insight='''insight
mindfulness
mental
feeling
mind
consciousness
contemplating
comprehension
enlightenment
reflection
knowledge
path
practice
concentration
reason
method
ignorance
anxiety
wisdom'''.split('\n')
insight = list(set(insight))

health='''health
body
suffering'''.split('\n')
health = list(set(health))

community='''community
people
world
elder
relations'''.split('\n')
community = list(set(community))

individual='''individual
self
subject
person
lives'''.split('\n')
individual = list(set(individual))

time='''time
state
process
activity'''.split('\n')
time = list(set(time))

In [92]:
cats=[insight,health,community,individual,time]
li=[]

for cat in cats:
    li.append(freq[freq['word'].isin(cat)])

In [93]:
cats_str=['insight','health','community','individual','time']

for no, cat in enumerate(cats_str):
    li[no]['cat']=cat
    
df=pd.DataFrame(columns=['word','count','freq','cat'])
df=df.append(li)

/Users/Michal_Palinski/miniconda3/envs/myPython/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [94]:
df['count']=df['count'].astype(int)
# df['freq']=df['freq'].round(1)

dfGr=df.groupby('cat').sum()
dfGr['keywords']=df.groupby('cat').size()
dfGr['count_per_keyword']=dfGr['count']/dfGr['keywords']
dfGr['count_per_keyword']=dfGr['count_per_keyword'].round(1)

In [95]:
frequency = {}
document_text = open('./texts/s_sutta.txt', 'r')
text_string = document_text.read().lower()
match_pattern = re.findall(r'\b[a-z]{3,15}\b', text_string)
 
for word in match_pattern:
    count = frequency.get(word,0)
    frequency[word] = count + 1
     
frequency_list = frequency.keys()

In [96]:
words_sum=sum(frequency.values())

In [97]:
dfGr['freq']=dfGr['count']/words_sum*100

In [98]:
dfGr

,count,freq,keywords,count_per_keyword
cat,,,,
community,2788,5.668625,5,557.6
health,3609,7.337901,3,1203.0
individual,4493,9.135270,5,898.6
insight,15415,31.342130,19,811.3
time,2213,4.499522,4,553.2


### Topic modeling

In [84]:
corpus=doc['text'].tolist()
corpusStr=' '.join(corpus)

In [85]:
corpus=[x.lower() for x in corpus]

In [86]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)
dtm_tf = tf_vectorizer.fit_transform(corpus)
print(dtm_tf.shape)

(3712, 3196)


In [87]:
tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(corpus)
print(dtm_tfidf.shape)

(3712, 3196)


In [88]:
# for TF DTM
lda_tf = LatentDirichletAllocation(n_components=15, random_state=0)
lda_tf.fit(dtm_tf)
# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(n_components=15, random_state=0)
lda_tfidf.fit(dtm_tfidf)

/Users/Michal_Palinski/miniconda3/envs/myPython/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:532: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
/Users/Michal_Palinski/miniconda3/envs/myPython/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:532: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=15, n_jobs=1,
             n_topics=None, perp_tol=0.1, random_state=0,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [162]:
prep=pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='tsne')

In [164]:
ht=pyLDAvis.prepared_data_to_html(prep)

In [168]:
 pyLDAvis.save_html(prep,'./docs/lda.html')

### Co-occurrence analysis

In [99]:
important=df['word'].tolist()

In [102]:
old_datetime = datetime.now()
ngrams = [(1,1)]
res = './res/sutta/'
month = 'test'

for ngram_art, ngram_compare in ngrams:
#     dataframe, sort by coef
#     take only these ngrams_compare which experience significant growth
    coefs_compare = pd.read_csv(res + 'freq_' + str(ngram_compare) + '-all.csv', index_col=0)
    coefs_compare = coefs_compare.loc[coefs_compare['freq'] < 0.5].sort_values('freq', ascending=False)
    
    coefs_art = pd.read_csv(res + 'freq_' + str(ngram_art) + '-all.csv', index_col=0)
    coefs_art = coefs_art.loc[coefs_art['freq'] < 0.5].sort_values('freq', ascending=False)

    # take only most common, growing ngrams
#     important_words = coefs_art.index[2:12].tolist()
    if ngram_art == 1:
        important_words = important
    elif ngram_art == 2:
        important_words = important_2
    
    cooc_all = []
    other_words = {}

    # 5000 most significant ngrams
    for i, row in coefs_compare[:2500].iterrows():
        other_words[i] = row['freq']
    
    cooc = {}
    print(month, datetime.now() - old_datetime)
    old_datetime = datetime.now()
        # use the fact that indices are the same
        # probably more clear would be joining by links
    
    dfs_art = pd.read_csv(res + str(ngram_art) + '.csv')
    dfs_art['text'] = dfs_art['text'].apply(lambda x: set(literal_eval(x)))
    dfs_art.rename(columns={'text': 'text_token_art'}, inplace=True)
    dfs_compare = pd.read_csv(res + str(ngram_compare) + '.csv')
    dfs_compare['text'] = dfs_compare['text'].apply(lambda x: set(literal_eval(x)))
    dfs_compare.rename(columns={'text': 'text_token_compare'}, inplace=True)
        
    dfs = pd.concat([dfs_art[['text_token_art']], dfs_compare[['text_token_compare']]], axis=1)
        
    article_word_count = {}
        
    # save RAM
    del dfs_art
    del dfs_compare
    gc.collect()
        
    for word in important_words:
        print(word)
        cooc[word] = {}
            # take only articles with the desired word
        df_site_cont = dfs.loc[dfs['text_token_art'].apply(lambda x: word in x)]
        article_word_count[word] = df_site_cont.shape[0]

        # we don't want iterating a lot of times over a dataframe
        # because it's slow, so first we assign zeroes to all words, and then add 1 if word exists
        for k in other_words.keys():
            cooc[word][k] = 0

        # iterate over articles, then after words
        for i, row in df_site_cont.iterrows():
            for k in other_words.keys():
                cooc[word][k] += k in row['text_token_compare']
                
        cooc_all.append(pd.DataFrame.from_dict(cooc))
    
    # concat
    cooc = pd.concat(cooc_all, axis=1)

    cooc = cooc.loc[:, ~cooc.columns.duplicated()]
    # three coefficients
    # frequency, count normalized by mean, count normalized by mean squared
    for i_word, word in enumerate(important_words):
        print('coef', word, 100*i_word/len(important_words), datetime.now() - old_datetime)
        old_datetime = datetime.now()
        if word not in cooc.columns:
            continue
        if article_word_count[word] > 0:
            cooc[word + '_freq'] = 100 * cooc[word] / article_word_count[word]
        else:
            cooc[word + '_freq'] = 0
                
    # count weighted values
    for i_word, word in enumerate(important_words):
        print('count', word, 100*i_word/len(important_words), datetime.now() - old_datetime)
        old_datetime = datetime.now()
        cooc[word + '_art_' + 'freq_weighted'] = 0
        cooc[word + '_art_' + 'freq_weighted_normalized'] = 0
        cooc[word + '_art_' + 'freq_weighted_normalized2'] = 0
        if word in cooc.columns: # avoid KeyErrors
            cooc[word + '_art_' + 'freq_weighted'] += cooc[word + '_freq']
            cooc[word + '_art_' + 'freq_weighted_normalized'] += cooc[word + '_freq']
            cooc[word + '_art_' + 'freq_weighted_normalized2'] += cooc[word +  '_freq']
        for word_compare, mean in other_words.items():
            cooc.loc[word_compare, word + '_art_' + 'freq_weighted_normalized'] = cooc.loc[word_compare, word + '_art_' + 'freq_weighted_normalized'] / mean
            cooc.loc[word_compare, word + '_art_' + 'freq_weighted_normalized2'] = cooc.loc[word_compare, word + '_art_' + 'freq_weighted_normalized2'] / mean**2

    cooc[[x for x in cooc.columns if 'weighted' in x]].to_csv(res + 'cooc' + str(ngram_art) + str(ngram_compare) + 'weighted_2.csv')

test 0:00:00.183511
wisdom
reason
path
anxiety
concentration
reflection
practice
contemplating
ignorance
knowledge
method
feeling
mental
enlightenment
comprehension
insight
consciousness
mind
mindfulness
body
health
suffering
community
relations
elder
people
world
person
individual
subject
lives
self
activity
state
time
process
coef wisdom 0.0 0:05:24.509058
coef reason 2.7777777777777777 0:00:00.159586
coef path 5.555555555555555 0:00:00.001589
coef anxiety 8.333333333333334 0:00:00.001093
coef concentration 11.11111111111111 0:00:00.000823
coef reflection 13.88888888888889 0:00:00.001126
coef practice 16.666666666666668 0:00:00.001123
coef contemplating 19.444444444444443 0:00:00.001084
coef ignorance 22.22222222222222 0:00:00.001203
coef knowledge 25.0 0:00:00.001020
coef method 27.77777777777778 0:00:00.000847
coef feeling 30.555555555555557 0:00:00.000840
coef mental 33.333333333333336 0:00:00.000980
coef enlightenment 36.111111111111114 0:00:00.000943
coef comprehension 38.888888

In [103]:
cooc=pd.read_csv('./res/sutta/cooc11weighted_2.csv')
cooc.rename(columns={'Unnamed: 0':'keywords'},inplace=True)

In [128]:
cooc.to_excel('suttas_cooc.xlsx')

In [140]:
suff=cooc.sort_values('suffering_art_freq_weighted_normalized',ascending=False)[:30]['keywords'].tolist()

In [143]:
body=cooc.sort_values('body_art_freq_weighted_normalized',ascending=False)[:30]['keywords'].tolist()

In [144]:
peop=cooc.sort_values('people_art_freq_weighted_normalized',ascending=False)[:30]['keywords'].tolist()

In [145]:
mind=cooc.sort_values('mind_art_freq_weighted_normalized',ascending=False)[:30]['keywords'].tolist()

In [156]:
coocs_chosen=pd.DataFrame(
    {'temp':range(30),'suffering_s': suff,
     'body_s': body,
     'people_s':peop,
     'mind_s':mind
    })

In [157]:
coocs_chosen.to_csv('coocs_chosen_suttas.csv')